##### MAIN OVERVIEW:
##### Written 2023/07 by Christina Lebonville
##### Last updated: 2023/08/10
###### For batch processing prediction export on all videos after finished model's inference.
###### REQUIRED KERNEL: "deg"

- Opens DEG Project
- Exports Predictions to CSV for All Videos 
- MUST use "python -m ipykernel install --user --name=deg" in anaconda command line to make the deg environment detectable in the notebook and then select the environment in the notebook before running the below code

In [1]:
import deepethogram
import logging
import multiprocessing
import os
import random
import h5py
from deepethogram import configuration, postprocessing, projects, utils, file_io
from deepethogram.debug import print_dataset_info

config_file='project_config.yaml'
data_path='H://TrueBout-Gen4_deepethogram/DATA'
project_path='H://TrueBout-Gen4_deepethogram/'


files = os.listdir(project_path)
assert 'DATA' in files, 'DATA directory not found! {}'.format(files)
assert 'models' in files, 'models directory not found! {}'.format(files)
assert 'project_config.yaml' in files, 'project config not found! {}'.format(files)

print_dataset_info(os.path.join(project_path, 'DATA'))
print('Project Loaded Successfully Unless Errors Reported Above')

C:\ProgramData\Anaconda3\envs\deg\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
3 UNLABELED ROWS!VIDEO WILL NOT BE USED FOR FEATURE_EXTRACTOR OR SEQUENCE TRAINING.
3 UNLABELED ROWS!VIDEO WILL NOT BE USED FOR FEATURE_EXTRACTOR OR SEQUENCE TRAINING.
3 UNLABELED ROWS!VIDEO WILL NOT BE USED FOR FEATURE_EXTRACTOR OR SEQUENCE TRAINING.
3 UNLABELED ROWS!VIDEO WILL NOT BE USED FOR FEATURE_EXTRACTOR OR SEQUENCE TRAINING.
3 UNLABELED ROWS!VIDEO WILL NOT BE USED FOR FEATURE_EXTRACTOR OR SEQUENCE TRAINING.
3 UNLABELED ROWS!VIDEO WILL NOT BE USED FOR FEATURE_EXTRACTOR OR SEQUENCE TRAINING.
3 UNLABELED ROWS!VIDEO WILL NOT BE USED FOR FEATURE_EXTRACTOR OR SEQUENCE TRAINING.


Project Loaded Successfully Unless Errors Reported Above


In [7]:
##Double-check a random record to make sure an entry for the two models exists
##in this case either ResNet18 (deg_f), ResNet50 (deg_m), or 3D-ResNet34 (deg_s) for Feature Extractor Model
##and tgmj for the Sequence Model

# this just parses our DATA directory, to get the path to each file for each video
records = projects.get_records_from_datadir(os.path.join(project_path, 'DATA'))
animal = random.choice(list(records.keys()))
record = records[animal]

# I call the file output by inference the `outputfile` in various places in the code
outputfile = record['output']

utils.print_hdf5(outputfile)

##Optional, you can check the probabilities for each model if you want - may be useful in future
# we use the h5py package for this
#with h5py.File(outputfile, 'r') as f:
#  probabilities = f['tgmj/P'][:]
#  thresholds = f['tgmj/thresholds'][:]

    # n frames x K behaviors
#print(probabilities.shape)

#print(thresholds)

resnet18
|--- P: (89278, 3) float32
|--- class_names: (3,) object
|--- flow_features: (89278, 512) float32
|--- logits: (89278, 3) float32
|--- spatial_features: (89278, 512) float32
|--- thresholds: (3,) float32
resnet50
|--- P: (89278, 3) float32
|--- class_names: (3,) object
|--- flow_features: (89278, 512) float32
|--- logits: (89278, 3) float32
|--- spatial_features: (89278, 512) float32
|--- thresholds: (3,) float32
tgmj
|--- P: (89278, 3) float32
|--- class_names: (3,) object
|--- logits: (89278, 3) float32
|--- thresholds: (3,) float32
attrs: 
(89278, 3)
[0.14078945 0.85931057 0.5       ]


In [2]:
cfg = configuration.make_postprocessing_cfg(project_path=project_path)
#cfg.sequence.output_name='resnet50' ##To export feature extractor predictions instead of sequence predictions
postprocessing.postprocess_and_save(cfg)
print('Success! Check your data path for updated prediction files.')

Success! Check your data path for updated prediction files.


In [4]:
# load a random record to see if predictions were successfully created
import pandas as pd
records = projects.get_records_from_datadir(os.path.join(project_path, 'DATA'))
animal = random.choice(list(records.keys()))
record = records[animal]
# figure out the filename
predictions_filename = os.path.join(os.path.dirname(record['rgb']), record['key'] + '_predictions.csv')
assert os.path.isfile(predictions_filename)

# read csv
df = pd.read_csv(predictions_filename, index_col=0)
# display outputs
df.head()

,background,C,NC
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0
